# TP cours - Comparez le comportement du lasso et de la régression ridge


expression gène du cancer = f(état du patient) 

In [15]:
data_path = "~/Documents/openclassroom/Fomation_ingenieur_ML/data/"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## sklearn module : 
from sklearn import model_selection 
from sklearn import preprocessing


In [16]:
df = pd.read_csv(data_path+"TP_1_prostate_dataset.txt", delimiter = "\t")
df.head()

,col,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [174]:
## DESIGN MATRICES : 
X = df.drop(["col", "lpsa","train"], axis = 1)
y = df.lpsa

## SPLIT DATA 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## STANDARDIZE : 
from sklearn.preprocessing import StandardScaler
my_std = preprocessing.StandardScaler()
my_std.fit(X_train)
X_train_std = my_std.transform(X_train)
X_test_std = my_std.transform(X_test)

# 1. Linear Regression : 

##### Model  : $\varepsilon \sim \mathcal{N}(0,\sigma^2)$
$$ y = X\beta + \varepsilon$$


##### Likelihood : 
$$ L = \prod_{i=1}^n \mathbb{P}[x^{(i)}, y^{(i)} | \beta ] $$
Max of log likelihood 
    $\Longleftrightarrow \underset{\beta \in  R^{p+1}}{\max} \left[  n \log(\frac{1}{\sigma \sqrt{2\pi}}) - \frac{1}{2\sigma^2} \sum_{i=1}^n  (f_{\beta}(x_i) - y_i )^2  \right]$
    $\Longleftrightarrow \underset{\beta \in  R^{p+1}}{\min} \left[ \sum_{i=1}^n  (f_{\beta}(x_i) - y_i )^2  \right]$


##### Least squares : 
Min of least squares : $ \underset{\beta \in  R^{p+1}}{\min} (y-X\beta)^T (y-X\beta)$ 

In [179]:
from sklearn import linear_model

lr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, 
                                   n_jobs=1, normalize=False)
lr.fit(X_train_std,y_train)
#y_pred = lr.predict(X_test_std)

LinearRegression(n_jobs=1)

Mean Square Error : ("erreur quadratique moyenne")
$$ MSE = \frac{1}{n} \sum^n_{i=1} (f(x_i) - y_i)^2 $$

In [185]:
MSE = ((y_pred-y_test)**2).mean()
MSE

0.49258454714253436

# Ridge Regression : 
Regularization of the coefficients : $ \|\beta \|_2^2 = 1$

The model is :
$$ y = X\beta + \varepsilon \text{ under constraint } \|\beta \|_2^2 = 1$$

KKT condition $\rightarrow$ lagrange multipliers $\rightarrow$ functionnal rewrites : 
$$\underset{\beta \in  R^{p+1}}{\min} (y-X\beta)^T (y-X\beta) + \alpha  \|\beta \|_2^2 $$

$\alpha$ is an hyperparameter $\rightarrow$ fitted with CV 

In [189]:
param_grid = {"alpha":np.logspace(-5,5,50)}

lm_ridge = linear_model.Ridge()

CV_ridge = model_selection.GridSearchCV(lm_ridge, param_grid= param_grid, cv = 5)
CV_ridge.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([1.00000000e-05, 1.59985872e-05, 2.55954792e-05, 4.09491506e-05,
       6.55128557e-05, 1.04811313e-04, 1.67683294e-04, 2.68269580e-04,
       4.29193426e-04, 6.86648845e-04, 1.09854114e-03, 1.75751062e-03,
       2.81176870e-03, 4.49843267e-03, 7.19685673e-03, 1.15139540e-02,
       1.84206997e-02, 2.94705170e-02, 4.71486636e-02, 7....
       7.90604321e-01, 1.26485522e+00, 2.02358965e+00, 3.23745754e+00,
       5.17947468e+00, 8.28642773e+00, 1.32571137e+01, 2.12095089e+01,
       3.39322177e+01, 5.42867544e+01, 8.68511374e+01, 1.38949549e+02,
       2.22299648e+02, 3.55648031e+02, 5.68986603e+02, 9.10298178e+02,
       1.45634848e+03, 2.32995181e+03, 3.72759372e+03, 5.96362332e+03,
       9.54095476e+03, 1.52641797e+04, 2.44205309e+04, 3.90693994e+04,
       6.25055193e+04, 1.00000000e+05])})